In [1]:
import json, glob
import numpy as np
import rasterio as rio
import rasterio.features as features
import rasterio.warp as warp
from osgeo import ogr, osr
from shapely.geometry import Polygon
from shapely.wkb import loads as wkb_loads
from shapely.ops import unary_union, polygonize


In [2]:
imagens = glob.glob('*.tif')

In [3]:
def open_raster(imagens):
    crs_affine_all = []
    img_crs_all = []
    geom_all = {}
    
    for img in imagens:
        
        img_open = rio.open(img)
        img_arr = img_open.read() 
        crs_affine_all.append(img_open.transform) 
        img_crs_all.append(img_open.crs) 
        
        name = img.split(".")[0]
        geom_all[name] = []
        names = [name]
        
        for geom, val in features.shapes(img_arr, transform = img_open.transform):

            geom = warp.transform_geom(img_open.crs, 'EPSG:4326', geom, precision=5)
            geom_all[name].append(geom)
    
    return geom_all, names

geom_all, names = open_raster(imagens)

In [4]:
def rasterio_to_shapely(geom_all):
    geom_ogr_all = []
    
    
    for key,pol in geom_all.items():
        
        for item in pol:
            
            json_str = json.dumps(item)
            geom_ogr = ogr.CreateGeometryFromJson(json_str)
            geom_ogr_all.append(geom_ogr)
            
    return geom_ogr_all

geom_ogr_all = rasterio_to_shapely(geom_all)

In [5]:
def export_pol_to_shapefile(geom_ogr_all):
    
    driver_pol = ogr.GetDriverByName("ESRI Shapefile")
    data_source_pol = driver_pol.CreateDataSource("mapbiomas_polygon.shp")


    target_pol = osr.SpatialReference()
    target_pol.ImportFromEPSG(4326)
    

    layer = data_source_pol.CreateLayer("geom_ogr_all", target_pol, ogr.wkbMultiPolygon)


    field_id = ogr.FieldDefn("id", ogr.OFTInteger)
    layer.CreateField(field_id)
    
    for count, item in enumerate(geom_ogr_all):
        

        area = item.Area()

        feature = ogr.Feature(layer.GetLayerDefn())
        feature.SetField("id", count+1)
        feature.SetGeometry(item)
        layer.CreateFeature(feature)
    
    
    layer.SyncToDisk()
    
export_pol_to_shapefile(geom_ogr_all)